# Implementación del Método Simplex Dual Phase

## Ejercicio a resolver: 

Minimizar 
$$Z = 5{x_1} - 4{x_2} + 3{x_3} $$
sujeto a:
$$
2x_1 + x_2 - x_3 = 10 
$$
$$
x_1 - 3x_2 + 2x_3 \ge 5
$$
$$
x_1 + x_2 + x_3  \le 15
$$
$$
x_1,x_2,x_3 \ge 0
$$


### Paso 1: Transformaciones

El primer paso es añadir las variables de holgura y exceso

Restricciones:
$$
2x_1 + x_2 - x_3 = 10 
$$
$$
x_1 - 3x_2 + 2x_3 - x_4 = 5
$$
$$
x_1 + x_2 + x_3 + x_5 = 15
$$
$$
x_1,x_2,x_3,x_4,x_5 \ge 0
$$

Donde se generó la variable de holgura $x_4$ y la variable de exceso $x_5$

## Paso 2: Minimizar las variables artificiales
Minimizar:
$$ S = x_4 + x_5$$
Sujeto a:
$$
2x_1 + x_2 - x_3 + R_1= 10 
$$
$$
x_1 - 3x_2 + 2x_3 - x_4 + R_2= 5
$$
$$
x_1 + x_2 + x_3 + x_5 = 15
$$
$$
x_1,x_2,x_3,x_4,x_5 \ge 0
$$
Se añadieron las variables artificales $R_1$ y $R_2$ para poder añadir variables básicas en las restricciones de $\ge$ y `=`

In [1]:
import numpy as np
import math
import pandas as pd

Se utilizará el método big M con M = 100 para las variables artificiales creadas

In [2]:
# Se representa de forma matricial cada uno de los coeficientes de las restricciones, incluyendo ambos las variables de holgura, exceso y 
# artificiales
# en el orden x1, x2, x3, x4, x5, R1, R2
M = 100
restricciones = np.array([[2,1,-1,0,0,1,0], #R1
                         [1,-3,2,-1,0,0,1], #R2
                         [1,1,1,0,1,0,0]],  #x5
                         dtype=float)
# vector de soluciones
soluciones  = np.array([10,5,15] ,dtype=float)
# vector de la función objetivo, también en orden x1, x2, x3, x4, x5, R1, R2
# Para usar el método de big M, se le asignan a R1 y R2 valores de 100
z = np.array([5,-4,3,0,0,-100,-100], dtype=float)

# Agregar columna de soluciones a la matriz
tabla = np.hstack((restricciones, soluciones.reshape(-1, 1)))
#Añadir la fila de z
tabla = np.vstack((tabla, np.append(z, 0)))

columnas = ['x1', 'x2', 'x3', 'x4', 'x5', 'R1', 'R2', 'solucion']
filas = ['R1', 'R2', 'x4', 'Z']

# Crear y mostrar el DataFrame
df = pd.DataFrame(tabla, columns=columnas, index=filas)
print(df)




     x1   x2   x3   x4   x5     R1     R2  solucion
R1  2.0  1.0 -1.0  0.0  0.0    1.0    0.0      10.0
R2  1.0 -3.0  2.0 -1.0  0.0    0.0    1.0       5.0
x4  1.0  1.0  1.0  0.0  1.0    0.0    0.0      15.0
Z   5.0 -4.0  3.0  0.0  0.0 -100.0 -100.0       0.0


El siguiente paso es recalcular la fila de Z, ya que actualmente es inconsistente con las variables básicas R1 y R2(-100 - 100 = 0)
#TODO justificar



$$Z_{nueva} = Z_{vieja} +  (M * filaR_1+ M*filaR_2)$$

$$ Z_{nueva} = (5,-4,3,0,0,-100,-100,0) +( 100\cdot(2,1,-1,0,0,1,0,10) + 100\cdot(1,-3,2,-1,0,0,1,5))


In [3]:
z_vieja = tabla[-1, :].copy()
z_nueva = z_vieja + M * tabla[0, :] + M * tabla[1, :]
tabla[-1, :] = z_nueva

df = pd.DataFrame(tabla, columns=columnas, index=filas)
print(df)

       x1     x2     x3     x4   x5   R1   R2  solucion
R1    2.0    1.0   -1.0    0.0  0.0  1.0  0.0      10.0
R2    1.0   -3.0    2.0   -1.0  0.0  0.0  1.0       5.0
x4    1.0    1.0    1.0    0.0  1.0  0.0  0.0      15.0
Z   305.0 -204.0  103.0 -100.0  0.0  0.0  0.0    1500.0


Como es un problema de minimización, se selecciona entre las columnas cuyo valor en la fila Z sean positivas. Entre las posibles, la columna de x1 es la que tiene el mayor valor en la columna Z, entonces esa será la columna pivote. 
- Fila de R_1: $10/2 = 5$
- Fila de R_2: $5/1 = 5$
- Fila de x_4: $15/1 = 15$

Como ambos $R_1$ y $R_2 $ tienen el mismo valor de dividir el valor de la columna de solución sobre el de la columna de $x_1$, se selecciona arbitrariamente que se reemplazará la fila de $R_1$

Primero se normalizará la fila de R1, dividiendo todo por 2

In [4]:
filaR1 = tabla[0, :].copy()
filaR1 = filaR1/2
tabla[0,:] = filaR1

#Nueva fila = (Fila actual) - (Coeficiente de la columna pivote) × (Nueva fila pivote)

# Actualizamos la fila Z
fila_Z_act = tabla[-1,:]
tabla[-1,:]  = fila_Z_act - (fila_Z_act[0]*filaR1)

# Actualizamos la fila R2
fila_R2_act = tabla[1,:]
tabla[1,:]  = fila_R2_act - (fila_R2_act[0]*filaR1)

# Actualizamos la fila x4
fila_x4_act = tabla[2,:]
tabla[2,:]  = fila_x4_act - (fila_x4_act[0]*filaR1)
#Entra la columna de x1 y reemplaza la de R1
filas = ['x1', 'R2', 'x4', 'Z']
df = pd.DataFrame(tabla, columns=columnas, index=filas)
print(df)



     x1     x2     x3     x4   x5     R1   R2  solucion
x1  1.0    0.5   -0.5    0.0  0.0    0.5  0.0       5.0
R2  0.0   -3.5    2.5   -1.0  0.0   -0.5  1.0       0.0
x4  0.0    0.5    1.5    0.0  1.0   -0.5  0.0      10.0
Z   0.0 -356.5  255.5 -100.0  0.0 -152.5  0.0     -25.0


Después de realizar el primer cambio, aún hay una columna con el coeficiente en la fila de Z positivo, entonces se realiza el mismo proceso para seleccionar la siguiente fila pivote con la columna pivote $x_3$:
- Fila de x_1: $5/-0.5 = -10$ NO SE CONSIDERA
- Fila de R_2: $0/2.5 = 0$ 
- Fila de x_4: $10/1.5 = 6.66$

Entonces se selecciona la fila de $R_2$ y se vuelve a hacer el mismo proceso



In [5]:
filaR2 = tabla[1, :].copy()
filaR2 = filaR2/2.5
tabla[1,:] = filaR2

#Nueva fila = (Fila actual) - (Coeficiente de la columna pivote) × (Nueva fila pivote)

# Actualizamos la fila Z
fila_Z_act = tabla[-1,:]
tabla[-1,:]  = fila_Z_act - (fila_Z_act[2]*filaR2)

# Actualizamos la fila x1
fila_x1_act = tabla[0,:]
tabla[0,:]  = fila_x1_act - (fila_x1_act[2]*filaR2)

# Actualizamos la fila x4
fila_x4_act = tabla[2,:]
tabla[2,:]  = fila_x4_act - (fila_x4_act[2]*filaR2)
#Entra la columna de x3 y reemplaza la de R2
filas = ['x1', 'x3', 'x4', 'Z']
df = pd.DataFrame(tabla, columns=columnas, index=filas)
print(df)

     x1   x2   x3   x4   x5     R1     R2  solucion
x1  1.0 -0.2  0.0 -0.2  0.0    0.4    0.2       5.0
x3  0.0 -1.4  1.0 -0.4  0.0   -0.2    0.4       0.0
x4  0.0  2.6  0.0  0.6  1.0   -0.2   -0.6      10.0
Z   0.0  1.2  0.0  2.2  0.0 -101.4 -102.2     -25.0


El siguiente paso es seleccionar la siguiente variable de la fila Z más positiva, en este caso siendo x4 como nuestra columna pivote. Seleccionamos la fila pivote de la misma manera que las otras 2 veces :)
- Fila de x_1: $5/-0.2 = -25$ NO SE CONSIDERA
- Fila de x_3: $0/-0.4 = -0$ NO SE CONSIDERA (tecnisismo del -0) 
- Fila de x_4: $10/0.6 = 16.66$

Se selecciona la fila pivote de $x_4$ 

In [ ]:
filax4 = tabla[2, :].copy()
filax4 = filax4 / tabla[2, 3] 

tabla[2, :] = filax4

# Nueva fila = (Fila actual) - (Coeficiente de la columna pivote) × (Nueva fila pivote)

# Actualizamos la fila Z
fila_Z_act = tabla[-1, :].copy()
tabla[-1, :] = fila_Z_act - (fila_Z_act[3] * filax4)  

# Actualizamos la fila x1
fila_x1_act = tabla[0, :].copy()
tabla[0, :] = fila_x1_act - (fila_x1_act[3] * filax4)

# Actualizamos la fila x3
fila_x3_act = tabla[1, :].copy()
tabla[1, :] = fila_x3_act - (fila_x3_act[3] * filax4)

# Entra y sale x4, no se alteran los nombres de las filas
filas = ['x1', 'x3', 'x4', 'Z']
df = pd.DataFrame(tabla, columns=columnas, index=filas)
print(df)

0.6000000000000001
     x1        x2   x3   x4        x5          R1     R2   solucion
x1  1.0  0.666667  0.0  0.0  0.333333    0.333333    0.0   8.333333
x3  0.0  0.333333  1.0  0.0  0.666667   -0.333333    0.0   6.666667
x4  0.0  4.333333  0.0  1.0  1.666667   -0.333333   -1.0  16.666667
Z   0.0 -8.333333  0.0  0.0 -3.666667 -100.666667 -100.0 -61.666667


Como ya todos los valores de la fila Z son negativos, llegamos a la solución óptima para R1 y R2 con 
$$x1 = 8.333$$
$$x2 = 6.67$$
$$x3 = 0$$
$$x4 = 16.67$$
:)
Ahora podemos eliminar las columnas de R1 y R2, ya que ya cumplieron su función